# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 
import sys
import os
import numpy as np
import pandas as pd
sys.path.append(os.getenv('PRICE_DATA'))

In [2]:
import os
import dask.dataframe as dd
price_data = os.getenv('PRICE_DATA')





c:\Users\USER\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\USER\AppData\Local\Temp\ipykernel_16420\786939951.py:2: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be r

In [3]:
##############
print(f"PRICE_DATA: {price_data}")  

if price_data is None:
    raise ValueError("PRICE_DATA environment variable is not set.")

PRICE_DATA: ../../05_src/data/prices/


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob


# Write your code below.

parquet_files = glob(os.path.join(price_data,"*/*.parquet/part.0.parquet"))

parquet_files


['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015.pa

In [10]:
price_data

'../../05_src/data/prices/'

In [11]:
(os.path.join(price_data,"*/*.parquet/part.0.parquet"))

'../../05_src/data/prices/*/*.parquet/part.0.parquet'

In [7]:
parquet_files

[]

In [13]:
ddf = dd.read_parquet(parquet_files)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [14]:

dd_feat = (ddf.groupby('ticker', group_keys=False)
        .apply(lambda x: x.assign(Close_lag_1=x['Close'].shift(1)))
        .assign(returns=lambda x: x['Close'] / x['Close_lag_1'] - 1)
        .assign(hi_lo_range=lambda x: x['High'] - x['Low'])
    )

C:\Users\USER\AppData\Local\Temp\ipykernel_16420\1198370255.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (ddf.groupby('ticker', group_keys=False)


In [15]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
HUM,2024-01-02,459.739990,471.549988,458.149994,470.100006,466.467133,1966900,Health Care,Managed Health Care,2024,NaN,NaN,13.399994
HUM,2024-01-03,474.910004,475.000000,463.839996,464.519989,460.930237,1307400,Health Care,Managed Health Care,2024,470.100006,-0.011870,11.160004
HUM,2024-01-04,469.100006,471.369995,466.170013,466.540009,462.934662,951800,Health Care,Managed Health Care,2024,464.519989,0.004349,5.199982
HUM,2024-01-05,464.029999,464.750000,453.730011,458.269989,454.728546,1692700,Health Care,Managed Health Care,2024,466.540009,-0.017726,11.019989
HUM,2024-01-08,458.980011,464.380005,451.989990,460.739990,457.179443,1584500,Health Care,Managed Health Care,2024,458.269989,0.005390,12.390015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2022-12-23,31.410000,31.690001,31.200001,31.670000,30.192837,2017500,Information Technology,Communications Equipment,2022,31.389999,0.008920,0.490000
JNPR,2022-12-27,31.740000,31.830000,31.510000,31.780001,30.297707,1644900,Information Technology,Communications Equipment,2022,31.670000,0.003473,0.320000
JNPR,2022-12-28,31.820000,31.879999,31.309999,31.360001,29.897295,1426300,Information Technology,Communications Equipment,2022,31.780001,-0.013216,0.570000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [16]:
# Write your code below.


dff = dd_feat.compute()


In [22]:
df_avg=(dff.groupby(['ticker']).apply(lambda x: x.assign(ma_return=x['returns'].rolling(window=10).mean())))

In [23]:
df_avg

Date        Open        High         Low       Close  \
ticker ticker                                                              
A      A      2017-01-03   45.930000   46.750000   45.740002   46.490002   
       A      2017-01-04   46.930000   47.380001   46.820000   47.099998   
       A      2017-01-05   47.049999   47.070000   46.360001   46.540001   
       A      2017-01-06   46.630001   48.070000   46.560001   47.990002   
       A      2017-01-09   48.009998   48.560001   47.910000   48.139999   
...                  ...         ...         ...         ...         ...   
ZTS    ZTS    2020-12-24  159.919998  161.869995  159.850006  160.720001   
       ZTS    2020-12-28  161.440002  162.830002  160.309998  162.389999   
       ZTS    2020-12-29  163.210007  165.100006  162.399994  163.119995   
       ZTS    2020-12-30  163.509995  164.800003  163.160004  164.270004   
       ZTS    2020-12-31  164.500000  165.679993  163.229996  165.500000   

                Adj Close   Volume       sector  \
ticker ticker                                     
A      A        43.824722  1739600  Health Care   
       A        44.399754  1821300  Health Care   
       A        43.871861  1503700  Health Care   
       A        45.238731  2883400  Health Care   
       A        45.380131  2575300  Health Care   
...                   ...      ...          ...   
ZTS    ZTS     156.067657   417400  Health Care   
       ZTS     157.689301  1522400  Health Care   
       ZTS     158.398163  1188400  Health Care   
       ZTS     159.514862  1009000  Health Care   
       ZTS     160.709274  1292600  Health Care   

                                    subsector  year  Close_lag_1   returns  \
ticker ticker                                                                
A      A       Life Sciences Tools & Services  2017          NaN       NaN   
       A       Life Sciences Tools & Services  2017    46.490002  0.013121   
       A       Life Sciences Tools & Services  2017    47.099998 -0.011890   
       A       Life Sciences Tools & Services  2017    46.540001  0.031156   
       A       Life Sciences Tools & Services  2017    47.990002  0.003126   
...                                       ...   ...          ...       ...   
ZTS    ZTS                    Pharmaceuticals  2020   159.850006  0.005443   
       ZTS                    Pharmaceuticals  2020   160.720001  0.010391   
       ZTS                    Pharmaceuticals  2020   162.389999  0.004495   
       ZTS                    Pharmaceuticals  2020   163.119995  0.007050   
       ZTS                    Pharmaceuticals  2020   164.270004  0.007488   

               hi_lo_range  ma_return  
ticker ticker                          
A      A          1.009998        NaN  
       A          0.560001        NaN  
       A          0.709999        NaN  
       A          1.509998        NaN  
       A          0.650002        NaN  
...                    ...        ...  
ZTS    ZTS        2.019989   0.001686  
       ZTS        2.520004   0.001833  
       ZTS        2.700012   0.002301  
       ZTS        1.639999   0.001464  
       ZTS        2.449997   0.003040  

[2778696 rows x 14 columns]

In [25]:
print(df_avg.describe())


                                Date          Open          High  \
count                        2778696  2.778696e+06  2.778696e+06   
mean   2013-02-01 14:24:10.957427968  7.929067e+01  8.020679e+01   
min              2000-01-03 00:00:00  3.020800e-02  3.052100e-02   
25%              2007-02-20 00:00:00  2.352000e+01  2.385000e+01   
50%              2013-06-06 00:00:00  4.410000e+01  4.461172e+01   
75%              2019-03-25 00:00:00  8.264000e+01  8.351000e+01   
max              2024-10-24 00:00:00  9.914170e+03  9.964770e+03   
std                              NaN  1.781350e+02  1.802268e+02   

                Low         Close     Adj Close        Volume          year  \
count  2.778696e+06  2.778696e+06  2.778696e+06  2.778696e+06  2.778696e+06   
mean   7.835441e+01  7.930263e+01  7.000281e+01  8.026106e+06  2.012589e+03   
min    2.697900e-02  3.052100e-02  3.052100e-02  0.000000e+00  2.000000e+03   
25%    2.319771e+01  2.353000e+01  1.664625e+01  9.295000e+05  2.007000

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

In [97]:
# Load necessary libraries
import os
from glob import glob
import dask.dataframe as dd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Load the PRICE_DATA environment variable
price_data_dir = os.getenv("PRICE_DATA")

# Use glob to find the path of all parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_dir, '*.parquet'))

# Read the parquet files into a Dask DataFrame
dd_df = dd.read_parquet(parquet_files)

# Add lags for variables Close and Adj_Close
dd_df['Close_lag'] = dd_df.groupby('ticker')['Close'].shift(1)
dd_df['Adj_Close_lag'] = dd_df.groupby('ticker')['Adj_Close'].shift(1)

# Calculate returns based on Adjusted Close
dd_df['returns'] = (dd_df['Adj_Close'] / dd_df['Adj_Close_lag']) - 1

# Calculate the hi-lo range
dd_df['hi_lo_range'] = dd_df['High'] - dd_df['Low']

# Assign the result to dd_feat
dd_feat = dd_df

# Convert the Dask DataFrame to a Pandas DataFrame
df = dd_feat.compute()

# Add a rolling average return calculation with a window of 10 days
df['rolling_avg_return'] = df.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(drop=True)

# Comments
# Was it necessary to convert to pandas to calculate the moving average return?
# Yes, in this case, we converted to a pandas DataFrame to utilize the .rolling() method. 
# Dask's rolling function operates differently and may not support all the operations in the same way.

# Would it have been better to do it in Dask? Why?
# Ideally, it would be better to perform the rolling average calculation in Dask 
# to leverage its parallel processing capabilities and avoid loading everything into memory.


ValueError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: empty urlpath sequence

In [98]:
print(price_data_dir)


../../05_src/data/prices/


In [99]:
parquet_files = glob(os.path.join(price_data_dir, '*.parquet'))
print(parquet_files)  # This should print a list of found parquet files


[]


In [102]:
print(f"Price data directory: {price_data_dir}")
if os.path.exists(price_data_dir):
    print("Directory exists.")
    print("Contents:", os.listdir(price_data_dir))
else:
    print("Directory does not exist.")



Price data directory: ../../05_src/data/prices/
Directory does not exist.


In [103]:
all_files = os.listdir(price_data_dir)
parquet_files = [f for f in all_files if f.endswith('.parquet')]
print(f"Found parquet files: {parquet_files}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../05_src/data/prices/'

In [104]:
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\USER\Documents\production\02_activities\assignments
